<a href="https://colab.research.google.com/github/wallik2/ML_algorithm_FromScratch/blob/main/%5BScratch%5D_4_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes from scratch 👶


![](https://miro.medium.com/max/551/0*xqTD8-GQ_FUYqYUQ.png)



---



# Dataset

In [33]:
import pandas as pd, re ,numpy as np

In [34]:
pos = [
            ["The hamburger is very delicious, I recommend it","positive"],
            ["The donut is too sweat and I like it","positive"],
            ["Hamburger is the best","positive"],
            ["Even some says it tastes like poop but I like its taste","positive"],
            ["The donut is not bad at all","positive"]
            ]

In [35]:
neg =[
      ["The taste of the fries is poop","negative"],
      ["Too much oil, I really not love it :)","negative"],
      ["I hate this restaurant, the worst ever","negative"],
      ["I saw a rat in my burger, what the heck?","negative"],
      ["The hamburger is really is Not as advertised","negative"]
      ]

In [36]:
test = [
    ["The burger is tastes like my pee","negative"],
["This restaurant is so dirty, I really really love it :)","negative"],
["This restaurant is good? ","negative"],
["I hate to say it, but this restaurant is too good to me ","positive"],
["I don’t see rat in my burger, so I give 5 stars ","positive"],
      ]

In [37]:
train = pd.DataFrame(data=pos+neg,columns=['reviews','sentiment']); test=pd.DataFrame(data=test,columns=['reviews','sentiment'])

In [38]:
display(train);display(test)

,reviews,sentiment
0,"The hamburger is very delicious, I recommend it",positive
1,The donut is too sweat and I like it,positive
2,Hamburger is the best,positive
3,Even some says it tastes like poop but I like ...,positive
4,The donut is not bad at all,positive
5,The taste of the fries is poop,negative
6,"Too much oil, I really not love it :)",negative
7,"I hate this restaurant, the worst ever",negative
8,"I saw a rat in my burger, what the heck?",negative
9,The hamburger is really is Not as advertised,negative


,reviews,sentiment
0,The burger is tastes like my pee,negative
1,"This restaurant is so dirty, I really really l...",negative
2,This restaurant is good?,negative
3,"I hate to say it, but this restaurant is too g...",positive
4,"I don’t see rat in my burger, so I give 5 stars",positive




---



In [39]:
pos_given = train[train['sentiment']=='positive'].copy()

neg_given = train[train['sentiment']=='negative'].copy()

In [40]:
pos_given

,reviews,sentiment
0,"The hamburger is very delicious, I recommend it",positive
1,The donut is too sweat and I like it,positive
2,Hamburger is the best,positive
3,Even some says it tastes like poop but I like ...,positive
4,The donut is not bad at all,positive


In [41]:
neg_given

,reviews,sentiment
5,The taste of the fries is poop,negative
6,"Too much oil, I really not love it :)",negative
7,"I hate this restaurant, the worst ever",negative
8,"I saw a rat in my burger, what the heck?",negative
9,The hamburger is really is Not as advertised,negative




---



Find probabiltiy of given positive and given negative:

## Tokenize the word

In [42]:
import re, numpy as np

def strip_off(string):
  string = string.lower()
  rx = r"[\w'(:))]+|\?+"
  lists = re.findall(rx,string)

  return lists

In [43]:
token_pos, token_neg = [],[]

for i in range(len(pos_given)):

  data = pos_given.iloc[i,0]
  token_pos += strip_off(data)


for i in range(len(neg_given)):

  data = neg_given.iloc[i,0]
  token_neg += strip_off(data)

In [44]:
#token_pos

#token_neg

Now find the the set of tokens, and its count in form of dictionary

In [45]:

pos_count,neg_count = 0,0
token_pos_count,token_neg_count = dict(),dict()

# let alpha = 1, add 1 count to each word
for i in set(token_pos):
  token_pos_count.update([(i,token_pos.count(i)+1)])
  pos_count += token_pos.count(i)+1

for i in set(token_neg):
  token_neg_count.update([(i,token_neg.count(i)+1)])
  neg_count += token_neg.count(i)+1

token_pos_count

{'all': 2,
 'and': 2,
 'at': 2,
 'bad': 2,
 'best': 2,
 'but': 2,
 'delicious': 2,
 'donut': 3,
 'even': 2,
 'hamburger': 3,
 'i': 4,
 'is': 5,
 'it': 4,
 'its': 2,
 'like': 4,
 'not': 2,
 'poop': 2,
 'recommend': 2,
 'says': 2,
 'some': 2,
 'sweat': 2,
 'taste': 2,
 'tastes': 2,
 'the': 5,
 'too': 2,
 'very': 2}

In [46]:
token_neg_count

{':)': 2,
 '?': 2,
 'a': 2,
 'advertised': 2,
 'as': 2,
 'burger': 2,
 'ever': 2,
 'fries': 2,
 'hamburger': 2,
 'hate': 2,
 'heck': 2,
 'i': 4,
 'in': 2,
 'is': 4,
 'it': 2,
 'love': 2,
 'much': 2,
 'my': 2,
 'not': 3,
 'of': 2,
 'oil': 2,
 'poop': 2,
 'rat': 2,
 'really': 3,
 'restaurant': 2,
 'saw': 2,
 'taste': 2,
 'the': 6,
 'this': 2,
 'too': 2,
 'what': 2,
 'worst': 2}

In [47]:
pos_df = pd.DataFrame(data=token_pos_count,index=[0])
neg_df = pd.DataFrame(data=token_neg_count,index=[0])

In [48]:
pos_df

,donut,says,some,like,delicious,very,all,i,too,hamburger,sweat,its,even,but,recommend,at,tastes,it,bad,is,and,poop,the,best,taste,not
0,3,2,2,4,2,2,2,4,2,3,2,2,2,2,2,2,2,4,2,5,2,2,5,2,2,2




---



To find the probability, must know the total number of word first 

In [49]:
print("token count of positive:", pos_count,"\ntoken count of negative:",neg_count)

token count of positive: 66 
token count of negative: 74


In [50]:
pos_prob_df = pos_df/pos_count
neg_prob_df = neg_df/neg_count

verify to make sure the prob is 1

In [51]:
pos_prob_df.sum(axis=1)

0    1.0
dtype: float64

In [52]:
neg_prob_df.sum(axis=1)

0    1.0
dtype: float64

So here's the probability of each word given positive




---




<fieldset> 

Ignore finding probabilities

In [53]:
#pos_prob_df

So here's the probability of each word given negative

In [54]:
#neg_prob_df



---



don't forget the prior probability of the label

In [55]:
prior = dict(train['sentiment'].value_counts())

In [56]:
p_pos,p_neg = prior['positive']/len(train),prior['negative']/len(train)


In [57]:
p_pos,p_neg

(0.5, 0.5)



---



Now we got the model yet, let's predict the test set

Let's recap our parameter of our training model

<fieldset>

**Positive**
- conditional probability is stored in *pos_prob_df*
- prior probability is stored in *p_pos*
- positive tokens counts : *pos_count*

**Negative**
- conditional probability is stored in *neg_prob_df*
- prior probability is stored in *p_neg*
- positive tokens counts : *neg_count*

In [58]:
pos_df

,donut,says,some,like,delicious,very,all,i,too,hamburger,sweat,its,even,but,recommend,at,tastes,it,bad,is,and,poop,the,best,taste,not
0,3,2,2,4,2,2,2,4,2,3,2,2,2,2,2,2,2,4,2,5,2,2,5,2,2,2


In [59]:
neg_df

,as,:),hate,a,ever,my,really,this,love,advertised,i,oil,in,restaurant,fries,?,too,hamburger,burger,much,of,saw,rat,what,worst,heck,it,is,poop,the,taste,not
0,2,2,2,2,2,2,3,2,2,2,4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,6,2,3


In [60]:

print("This is the probability of given by each label of the first reviews in test set")

print('test set',test.reviews[0],'\n\n')

print('pos of the first test text',0.5*6*1*6*3*5*1*1/73**7)

print('neg of the first test text',0.5*7*3*5*1*1*3*1/81**7)

This is the probability of given by each label of the first reviews in test set
test set The burger is tastes like my pee 


pos of the first test text 2.4440143037590848e-11
neg of the first test text 6.884706425084736e-12




---



In [61]:
test['Predicted_sentiment'] = np.nan;test

,reviews,sentiment,Predicted_sentiment
0,The burger is tastes like my pee,negative,NaN
1,"This restaurant is so dirty, I really really l...",negative,NaN
2,This restaurant is good?,negative,NaN
3,"I hate to say it, but this restaurant is too g...",positive,NaN
4,"I don’t see rat in my burger, so I give 5 stars",positive,NaN


if the word in test set exists, then simply multiply by the prbability of that word of the training set, otherwise the probability of that word is 1/(count of words given the label + total count in that reviews )

In [62]:
count = -1

for i in test['reviews']:
  text = strip_off(i) 
  count+=1
  print(i)
  #POSITIVE


  product_pos = 1 * p_pos
  for j in text:

    #print('len',len(text))
    try:
    # If it exists
      #print('try',pos_prob_df[j])
      hw = pos_df[j]+1
      #print(f'{j} with prob {pos_prob_df[j]}')
    except:
    # If it doesn't exists
      #print('except',1/((pos_count)+len(text)))
      hw = 1
      #print(f'{j} with prob', 1/((pos_count)+len(text)))

    product_pos *= hw/((pos_count)+len(text))

  print('pos:',product_pos[0])

   

  #NEGATIVE


  product_neg = 1 * p_neg
  for j in text:

    try:
      # If it exists
      hw = neg_df[j]+1
      #print(f'{j} with prob {neg_prob_df[j]}')
    except:
      # If it doesn't exists
      hw = 1
      #print(f'{j} with prob', 1/((neg_count)+len(text)))

    product_neg *= hw/((neg_count)+len(text))

  print('neg:',product_neg[0])
  print('\n')

# now predict by comparing which label has larger probability

  if product_pos[0] > product_neg[0]:
      test.iloc[count,2] = 'positive' 
    
  elif product_pos[0] < product_neg[0]:
      test.iloc[count,2] = 'negative'

  else: 
    #Extremely rare to happen especially when having big dataset
      test.iloc[count,2] = 'positiveXXnegative'


The burger is tastes like my pee
pos: 2.444014303759084e-11
neg: 6.8847064250847335e-12


This restaurant is so dirty, I really really love it :)
pos: 1.3294233011394842e-19
neg: 2.904210467567041e-17


This restaurant is good? 
pos: 1.6627597806992997e-09
neg: 2.19365494964397e-08


I hate to say it, but this restaurant is too good to me 
pos: 1.4459443748000367e-22
neg: 1.8568037415117153e-22


I don’t see rat in my burger, so I give 5 stars 
pos: 2.6776747681482158e-24
neg: 6.189345805039051e-23




In [63]:
test

,reviews,sentiment,Predicted_sentiment
0,The burger is tastes like my pee,negative,positive
1,"This restaurant is so dirty, I really really l...",negative,negative
2,This restaurant is good?,negative,negative
3,"I hate to say it, but this restaurant is too g...",positive,negative
4,"I don’t see rat in my burger, so I give 5 stars",positive,negative
